In [ ]:
!git clone https://github.com/OpenGVLab/InternImage.git # Contains some custom configs and modifications to training code


In [ ]:
%cd /content/InternImage
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113  -f https://download.pytorch.org/whl/torch_stable.html
!pip install -U openmim
!mim install mmcv-full==1.5.0
!pip install timm==0.6.11 mmdet==2.28.1
!pip install git+https://github.com/deepglint/unicom.git
%cd /content/InternImage/detection/ops_dcnv3
!sh ./make.sh

In [ ]:
%cd /content/InternImage
! python train.py configs/coco/dino_4scale_internimage_l_1x_coco_layer_wise_lr.py
# Train until satisfactory loss


In [ ]:
! python test.py configs/coco/dino_4scale_internimage_l_1x_coco_layer_wise_lr.py work_dirs/dino_4scale_internimage_l_1x_coco_layer_wise_lr/epoch_1.pth --out "\content\testt.pkl"

In [ ]:
import torch
import os
from PIL import Image
from torch.nn.functional import normalize
import unicom
import torchvision.transforms as transforms
import pickle
import tqdm
import pandas as pd

conf_threshold = 0.5
with open("/content/testt.pkl", "rb") as f:
    img_data = pickle.load(f)

img_dir = "/content/Test"
anchor_dir = "/content/crops"
conf_threshold = conf_threshold


torch.manual_seed(2048)
model, preprocess = unicom.load("ViT-L/14@336px")
model.eval().cuda()
transform = transforms.Compose([preprocess])



def get_imgs(idx):
    global img_data
    img_name = f'image_{idx:04}'
    base_img = Image.open(os.path.join(img_dir, f'{img_name}.png')).convert('RGB')

    anchor_img = Image.open(os.path.join(anchor_dir, f'{img_name}.png'))
    anchor_img = transform(anchor_img)

    t, coords = [anchor_img], []

    for j, img_info in enumerate(img_data[idx][0]):
        if j == 4:
            break  # Faster than slicing
        top_x, top_y, width, height, box_conf = img_info

        if box_conf >= conf_threshold:
            x_min = top_x / 1280
            y_min = top_y / 720
            x_max = width / 1280
            y_max = height / 720
            coords.append((conf_threshold, y_min, x_min, y_max, x_max))

            crop_img = base_img.crop((top_x, top_y, width, height))
            crop_img = transform(crop_img)
            t.append(crop_img)

    # print(len(t))
    return torch.stack(t, dim=0).cuda().squeeze(), coords, img_name

l = [["Image_ID", "class", "confidence", "ymin",  "xmin",  "ymax",  "xmax"]]

for i in tqdm.tqdm(range(1600)):
    imgs, coords, img_name = get_imgs(i)
    emb = normalize(model(imgs))
    cos_sim = emb[0] @ emb.T
    sus = torch.argmax(cos_sim[1:], dim=-1)

    for idx, coord in enumerate(coords):
        is_sus = 0 if sus != idx else 1
        l.append((img_name, is_sus, *coords[idx]))

df = pd.DataFrame(l)
df.to_csv("C:\\Users\\ricky\\Downloads\\submission.csv", index=False, header=False)
